## 動的計画法
全状態を探索すると計算時間が膨大になるから（同じ計算を繰り返すことによる）、一度計算した部分を保存しておいて再利用（メモ化）
どんな問題も無限の資源があれば全組み合わせを試して解くことができるが、それらの中には部分的に同じものがあるので、そういった繰り返しを避けられるのがメリット  
DPのDはDynamic ArrayのDか  

In [17]:
#01
"""
引数の組は高々品物の数と重さの積通り
i番目以降の品物から選んだ時、重さがjを超えない最大の価値
品物を詰めるたびにjをその品の重さだけ減らす
よくある方法では、i番目まで選べるとき重さがjを超えない最大価値みたいにDPを組むが、
やってること自体にそんなに差はない。DPはいろんな置き方があるのでいろいろできるといいか。
でもいつもの２重forのほうが分かりやすい

"""
N,W = map(int, input().split())
wv = [list(map(int, input().split())) for _ in range(N)]

dp = [[-1 for _ in range(W+1)] for _ in range(N+1)]

# i番目以降から重さの総和がｊ以下になるよう選ぶ
def solve(i,j):
    # 探索済み
    print("--", i,j)
    if dp[i][j] >= 0:
        return dp[i][j]
    res = 0
    # 全部調べた
    if i == N:
        res = 0
    # i番目のwが制限より大きいので選べない
    elif  j < wv[i][0]:
        # 次以降の品をみる
        res = solve(i+1, j)
    else:
        # i番目を入れる場合と入れない場合の両方の大きいほうについてみて大きいほうを選ぶ
        res = max(solve(i+1, j), solve(i+1, j - wv[i][0]) + wv[i][1])
    dp[i][j] = res
    return dp[i][j]

solve(0,W)
print(dp[0][W])
print(dp)

4 5
2 3
1 2
3 4
2 2
-- 0 5
-- 1 5
-- 2 5
-- 3 5
-- 4 5
-- 4 3
-- 3 2
-- 4 2
-- 4 0
-- 2 4
-- 3 4
-- 4 4
-- 4 2
-- 3 1
-- 4 1
-- 1 3
-- 2 3
-- 3 3
-- 4 3
-- 4 1
-- 3 0
-- 4 0
-- 2 2
-- 3 2
7
[[-1, -1, -1, -1, -1, 7], [-1, -1, -1, 4, -1, 6], [-1, -1, 2, 4, 4, 6], [0, 0, 2, 2, 2, 2], [0, 0, 0, 0, 0, 0]]


In [23]:

"""
２重for ver
漸化式でやるやつ
個人的にこっちの方がメモ化より好き
"""
N,W = map(int, input().split())
wv = [list(map(int, input().split())) for _ in range(N)]

dp = [[0 for _ in range(W+1)] for _ in range(N)]

for i in range(N):
    for j in range(W+1):
        # i番目の品の重さが制限を超えるなら選べない
        if wv[i][0] > j:
            if i == 0:
                dp[i][j] = 0
            else:
                dp[i][j] = dp[i-1][j]
        else:
            # i番目を選んだ場合と選ばなかった場合の大きい方
            dp[i][j] = max(dp[i-1][j], dp[i-1][j - wv[i][0]] + wv[i][1])
            
print(dp[N-1][W])
print(dp)

4 5
2 3
1 2
3 4
2 2
7
[[0, 0, 3, 3, 3, 3], [0, 2, 3, 5, 5, 5], [0, 2, 3, 5, 6, 7], [0, 2, 3, 5, 6, 7]]


## 最長共通部分列（LCS）問題
典型問題なのでマスターしたい。  
dp(i,j)：Sのi文字目までとTのj文字目までのLCSの長さ  
とおくと、  
S(i)=T(j)：既存のLCSにS(i+1)を加えたもの  
あるいは  
Sのi+1文字目までとTのj文字目までのLCSの長さ  
か  
Sのi文字目までとTのj+1文字目までのLCSの長さ  
が最長になる  
if S(i) == T(j):  
    dp(i+1,j+1) = dp(i,j) + 1  
else:  
    dp(i+1,j+1) = max( dp(i+1,j), dp(i,j+1) )  
      
みたいな  
back trackingによる復元とかもあるがこれもちょっと難しい  

In [2]:
S = input()
T = input()

#dp[i+1][j+1]：sのi文字目までとdのj文字目まで見たときのLCSの長さ
dp = [[0 for i in range(len(T)+1)] for _ in range(len(S)+1)]

# それぞれの先頭から一文字ずつ増やしたときのLCSの長さを確認していく
# len(S)*len(T)
for i in range(len(S)):
    for j in range(len(T)):
        if S[i] == T[j]:
            dp[i+1][j+1] = dp[i][j] + 1
        else:
            dp[i+1][j+1] = max(dp[i][j+1], dp[i+1][j])
            
print(dp[len(S)][len(T)])

abcd
becd
3


## 制限なしナップザック問題
今まで：i番目までの商品を選べる時、jまで鞄に入れられるなら    
制限なし：上に加えて、それぞれk個までいれるかも  
→i*j*kの３重ループ  
  
問題は一種類につき何個でも選べること。  
dp[i][j]:dp[i][j-w]+v[i]で、一個ずつ増やすみたいな形でkのループを消す。

In [4]:
N = int(input())
W = int(input())
wv = [list(map(int, input().split())) for _ in range(N)]

dp = [[0 for _ in range(W+1)] for _ in range(N)]

# 商品一つ目からN個目まで
for i in range(N):
    # 0からWまで
    for j in range(W+1):
        # 制限を超える場合はその商品を選べないのでその商品を選ばない状態での最大を選択
        if wv[i][0] > j:
            if i == 0:
                dp[i][j] = 0
            else:
                dp[i][j] = dp[i-1][j]
        else:
            # i番目の価値が小さいとi-1番目までで作る方が価値が高い場合があるのでちゃんと比較
            # ここもi=0かどうかでやらないといけないのでこの書き方はまずい
            dp[i][j] = max(dp[i-1][j], dp[i][j - wv[i][0]] + wv[i][1])
            
print(dp[N-1][W])

3
7
3 4
4 5
2 3
10


## N*Wが大きいナップザック問題
こういう場合は重さに対する価値を最大にするのではなく、価値に対する重さを最小にするとうまくいく  
*最小と最大は切り替えられる*  
dp(i,j):価値の総和がjになるように選んだ時の最小の重さ


In [19]:
N = int(input())
W = int(input())
INF = float("inf")
wv = [list(map(int, input().split())) for _ in range(N)]
V = 100
dp = [[INF for _ in range(N*V + 1)] for _ in range(N+1)]
dp[0][0] = 0

for i in range(N):
    for j in range(N*V+1):
        
        if j < wv[i][1]:
            # i番目の価値がjを超えるので選べない
            dp[i+1][j] = dp[i][j]
        else:
            dp[i+1][j] = min(dp[i][j], dp[i][j - wv[i][1]] + wv[i][0])

            
# N個全部選べる時の重さがWを超えない最大価値を探す
# 価値はインデックスに対応していて要素がその時の最小重さなので注意
ans = 0
for i in range(len(dp[N])):
    if dp[N][i] <= W:
        ans = i
    
        
print(ans)
print(dp[N])

4
5
2 3
1 2
3 4
2 2
7
[0, inf, 1, 2, 3, 3, 4, 5, 6, 6, inf, 8, inf, inf, inf, inf, inf, inf, inf, inf, inf]


In [2]:
N,W = map(int, input().split())
wv = [list(map(int, input().split())) for _ in range(N)]
INF = float("inf")
V = 100 # 価値の最大値
dp = [[ INF for _ in range(N*W+1)] for _ in range(N+1)]
dp[0][0] = 0

for i in range(N):
    for j in range(N*W+1):
        if wv[i][1] > j:
            continue
        else:
            dp[i+1][j] = min(dp[i][j], dp[i][j - wv[i][1]] + wv[i][0])

ans = 0
for i in range(len(dp[N])):
    if dp[N][i] <= W:
        ans = i
        
print(ans)

4 5
2 3
1 2
3 4
2 2
7


## 個数制限付きナップザック


dp(i+1,j):i番目まででｊを作る際に余るi番目の個数（作れないなら-1）  

i-1まででｊができるならまるまるiを残せる。j-a_iができるなら（そしてそこまででa_iが一つでも残ってるなら）jができる、みたいに考えていける。ｊを０から増やしていって判断。a_iを残せる個数が漸化式の値になるのは変な感じもするが、値が負でなければjを作れる→漸化式の値でなくインデックスも利用して情報を渡していくみたいな？？？

In [5]:
N = int(input())
A = list(map(int, input().split()))
M = list(map(int, input().split()))
K = int(input())

dp = [[-1 for _ in range(K+1)] for _ in range(N+1)]
dp[0][0] = 0

for i in range(N):
    for j in range(K+1):
        # a_iなしでｊができるので丸々残せる
        # elifの条件より後にしてしまうとi-1番目まででｊ作れるのにそれを使わないことになるので注意
        if dp[i][j] >= 0:
            dp[i+1][j] = M[i]
            
        # a_iがそもそもｊを超えているor j - a_iを作るとa_iが足りなくなる
        elif A[i] > j or dp[i+1][j - A[i]] <= 0:
            continue
        # j - a_i　に a_iを一つ足してｊを作るので、a_iの余り個数が一つ減る
        else:
            dp[i+1][j] = dp[i+1][j - A[i]] - 1
            
print(dp[N][K] >= 0)
print(dp)

"""
バックトラッキングをやるなら、
dp[N][K]を見る→N番目が何個余るか（N番目を何個使うか）がわかる
→dp[N-1][K-a_N-1*N-1番目を使った数]を見る→N-1番目が何個余るか（N-1番目を何個使うか）がわかる
→・・・
で何を何個使うかを調べられるはず
"""

3
3 5 8
3 2 2
17
True
[[0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [3, -1, -1, 2, -1, -1, 1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1], [2, -1, -1, 2, -1, 1, 2, -1, 1, 2, 0, 1, -1, 0, 1, -1, 0, -1], [2, -1, -1, 2, -1, 2, 2, -1, 2, 2, 2, 2, -1, 2, 2, -1, 2, 1]]


## 最長増加部分列

dp(i)：最後がa_iであるような最長の増加部分列の長さ  
とすると、そのような増加部分列は  
a_iのみか、それまでの最長増加部分列にa_iを付け足したもの  
となるので、  
dp(i) = max(1, dp(j)+1)   (j<i でa_j < a_iの時後者)  
これはO(N**2)(i,j　で二重for回す)
  
最小部分列の最後尾の値が小さいほど続けやすいので、  
dp(i)：長さがiの最小部分列の最終要素の値の最小値（なければINF）    
でもできる


In [1]:
from bisect import bisect_left
N = int(input())

A = list(map(int, input().split()))

INF = float("inf")
dp = [INF] * N



for i in range (N):
    # 一つ目はそのまま最長増加部分列になる
    if i == 0:
        dp[i] = A[i]
    else:
        # 最長増加部分列の現時点の最後のやつより小さい値は、どこか知ら入れる場所（更新できるところ）があるのでbisectで探して入れる
        if dp[i-1] > A[i]:
            idx = bisect_left(dp, A[i])
            dp[idx] = A[i]
        # 最長増加部分列の最後より大きい値が出てきたらそれの末尾に追加できる
        elif dp[i-1] < A[i]:
            dp[i] = A[i]
            
ans = 0
for i in range(N):
    if dp[i] != INF:
        ans += 1
        
print(ans)
print(dp)

"""
input

5
4 2 3 1 5
"""

5
4 2 3 1 5
3
[1, 3, 5, inf, inf]


## 分割数
http://drken1215.hatenablog.com/entry/2018/01/16/222843  

P(n,k)：nをk個以下の１以上の自然数の和で表したときの場合の数  
とおくと、  
P(n,k) = P(n-k,k) + P(n,k-1)


### 一項目の導出
１項目は０を含まないｋ個の自然数でｎを表す場合にあたる。  
Pの条件が、１以上の自然数の和で表す、なので、まず先にk個分１を割り振っておいて、そのあと残りのn-kをどのように割り振るか決める。これがP(n-k,k)


### 二項目の導出
こちらは０を含む場合を表す。ｋ個の自然数（０を含む）でｎを作るとき、ある一つのゼロを取り除いたら、k-1個の自然数でｎを作っているのと同じになる。ということで、P(n,k-1)


何個以上でもよい１以上への分割ならP(n,n)で。こいつはO(n√n)で求まるらしい



In [10]:
N,K= map(int, input().split())
M = 10**9 + 7

dp = [[0 for _ in range(K+1)] for _ in range(N+1)]
# 
dp[0][0] = 1
# iをj個に分割
for i in range(N+1):
    for j in range(K+1):
        # 分割できない場合、j-1個に分割するときと同じ
        if i < j:
            dp[i][j] = dp[i][j-1]
        else:
            dp[i][j] = dp[i-j][j] + dp[i][j-1]
            
            
print(dp[-1][-1] % M)
print(dp)

7 7
15
[[1, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 2, 2, 2, 2, 2, 2], [0, 1, 2, 3, 3, 3, 3, 3], [0, 1, 3, 4, 5, 5, 5, 5], [0, 1, 3, 5, 6, 7, 7, 7], [0, 1, 4, 7, 9, 10, 11, 11], [0, 1, 4, 8, 11, 13, 14, 15]]


In [ ]:
## 重複組み合わせ
#https://emtubasa.hateblo.jp/entry/2018/08/29/161456
難しいので復習ちゃんとする
n種類の品、i番目はa_i個ある、これらからｍ個えらぶ場合の数

同じ種類のものは区別できない

dp(i+1,j)：i番目までからj個選ぶ場合の数  
とすると、i-1までからj-k、i番目からk個選べばよい。 -> dp(i+1,j) = sigma(dp(i, j-k))  （k:0～j）
ここで、i-1までからj-k個えらぶdp(i, j-k)は、j-kをj-k-1にずらしてシグマの外で帳尻を合わせることを考える。

ずらす前にあったやつ：k=0の時のdp(i,j)を加える
ずらして出てきたやつ：k=j(かa_iの小さい方)のdp(i, j-min(j, a_i)-1)を取り除くことで、

dp(i+1,j) = sigma(dp(i, j-k)) = sigma(dp(i, j-k-1))   + dp(i,j) - dp(i, j-min(j, a_i)-1)  

ここで、右辺の１項目について考える。  
k=j (min(j, a_i) = j)のとき、dp(i, j-j-1)=0  
k=a_iのとき、j > a_iなのでk=jとならない  
よってｊをj-1に置き換えてもよく、そうすると１項目は  dp(i+1, j-1)となる

In [16]:
N,M = map(int, input().split())
A = list(map(int, input().split()))

mod = 10**9+7

dp = [[0 for _ in range(M+1)] for _ in range(N+1)]

for i in range(N+1):
    #i番目を一つも選ばない方法は一種類のみ
    dp[i][0] = 1
    
for i in range(N):
    # j = 0を上で作ったので飛ばすかたち
    for j in range(1, M+1):
        # 上のメモの３項めが０以下の場合は考慮しない
        if j - A[i] -1 < 0:
            dp[i+1][j] = dp[i+1][j-1] + dp[i][j]
        else:
            dp[i+1][j] = dp[i+1][j-1] + dp[i][j] - dp[i][j - A[i] - 1]
            print(i,j,j - A[i] - 1)
            
print(dp[-1][-1])
print(dp)

3 3
1 2 3
0 2 0
0 3 1
1 3 0
6
[[1, 0, 0, 0], [1, 1, 0, 0], [1, 2, 2, 1], [1, 3, 5, 6]]


以下類題

In [ ]:
"""
Typical DP Contest A

典型的なら制限内で最大の～とかになりそうだが、これは得点の個数。
i番目の問題までで作れる得点をi+1に移しつつ、i+1番目の得点ををそれぞれの点に加える。
よくあるdpぽい式にしたが、キューを使ってもうまくいくと思う（i番目まででできた得点をキューに持っておいて、i+1番目の点をそれぞれに足して新たな点を作るみたいな）
"""


N = int(input())

p = list(map(int, input().split()))



dp = [[0 for _ in range(sum(p)+1)] for _ in range(N+1)]
dp[0][0] = 1

for i in range(N):
    for j in range(sum(p)+1):
        # i番目の問題まででｊ点が作れるならdp[i][j] = 1
        if dp[i][j] == 1:
            dp[i+1][j] = 1 # ここでi番目の時の情報がi+1に引き継がれる
            dp[i+1][j+p[i]] = 1


print(sum(dp[-1]))

In [ ]:
"""
Typical DP Contest D

愚直に全探索すると絶対に間に合わない
i-1回さいころを振った時に手持ちの素因数2,3,5がそれぞれa,b,c個ある確率をdp[i][a][b][c]として、
i回目のさいころの目が2だったらそれぞれをa+1,b,c個持つことになり、
その場合の数dp[i+1][a+1][b][c]はdp[i][a][b][c]を利用して求まる、みたいなことを繰り返す

TODO：結構難しかったのでまたやる
"""

from collections import Counter


def main():
    N,D = map(int, input().split())

    num = D
    i = 2
    tmp = []
    for i in [2,3,5]:
        while num % i == 0:
            num = num / i
            tmp.append(i)

    if num > 1:
        print(0)
        exit()

    cntr = Counter(tmp)

    #さいころを投げて、2の素因数がa,3の～がｂ、５の～がｃ個ある場合の数 (i回さいころをふった時に)
    dp = [[[[0.0 for _ in range(cntr[5]+1)] for _ in range(cntr[3]+1)] for _ in range(cntr[2]+1)] for _ in range(N+1)]
    dp[0][0][0][0] = 1.0

    # 2がcntr[2]回以上出る場合の確率は全部cntr[2]の所に入れる
    # 最終的にDの倍数となる確率が分かればいいので。
    # でないと最後にそれぞれをcntr[2],cntr[3],cntr[5]個以上含むDの倍数になる確率を全て足し合わせる必要がある
    for i in range(N):
        for a in range(cntr[2]+1):
            for b in range(cntr[3]+1):
                for c in range(cntr[5]+1):
                    # 以下１～６までが出たときのdpの変化

                    # １が出ても何も変わらない
                    dp[i+1][a][b][c] += dp[i][a][b][c]/6.0
                    # ２が出たらaが一つ増える感じ
                    dp[i+1][min(a+1, cntr[2])][b][c] += dp[i][a][b][c]/6.0
                    # 3
                    dp[i+1][a][min(b+1, cntr[3])][c] += dp[i][a][b][c]/6.0
                    # 4は２が２個なので
                    dp[i+1][min(a+2, cntr[2])][b][c] += dp[i][a][b][c]/6.0
                    # 5
                    dp[i+1][a][b][min(c+1, cntr[5])] += dp[i][a][b][c]/6.0
                    # 6は２と３が一個ずつ
                    dp[i+1][min(a+1, cntr[2])][min(b+1, cntr[3])][c] += dp[i][a][b][c]/6.0

    print(dp[-1][cntr[2]][cntr[3]][cntr[5]])

if __name__ == "__main__":
    main()

In [ ]:

"""
AtCoder Beginner Contest 015 D

使用枚数Kと幅Wの二つの制限がある
n番目まで使える時に、k枚選んで幅wとなる最大価値をdpで求めると、
N*K*Wが 最大で25000000となり間に合わない

これを、n番目まで使える時に、k枚選んで価値bとなる最小幅でdpを考えると、
N*K*(N*B)で、最大12500000なので間に合うかも。
pypy3なら間に合った
"""

def main():
    W = int(input())
    N,K = map(int, input().split())

    A = [0] * N
    B = [0] * N
    for i in range(N):
        A[i], B[i] = map(int, input().split())

    total = sum(B)
    INF = float("inf")
    dp = [[[INF for _ in range(total+1)] for _ in range(K+1)] for _ in range(N+1)]

    dp[0][0][0] = 0


    for p in range(N+1):
        for q in range(K+1):
                dp[p][q][0] = 0


    # p番目の画像までから、q枚選んで価値がrになるときの最小幅
    for p in range(N):
        for q in range(1,K+1):
            for r in range(total+1):

                
                if B[p] > r:
                    dp[p+1][q][r] = dp[p][q][r]
                else:
                    dp[p+1][q][r] = min(dp[p][q][r], dp[p][q-1][r - B[p]] + A[p])

    ans = 0
    for i in range(total+1):
        if dp[N][K][i] <= W:
            ans = max(ans,i)
    print(ans)
    #print(dp[0])
    #print(dp[0][0][0])


if __name__ == "__main__":
    main()

In [ ]:
"""
dp配列を現在と次だけ持つようにすればpython3でもまにあった
"""

def main():
    W = int(input())
    N,K = map(int, input().split())

    A = [0] * N
    B = [0] * N
    for i in range(N):
        A[i], B[i] = map(int, input().split())

    total = sum(B)
    INF = float("inf")
    #dp = [[[INF for _ in range(total+1)] for _ in range(K+1)] for _ in range(2)]
    now = [[INF for _ in range(total+1)] for _ in range(K+1)]
    next = [[INF for _ in range(total+1)] for _ in range(K+1)]

    now[0][0] = 0


    for q in range(K+1):
        now[q][0] = 0
        next[q][0] = 0


    # p番目の画像までから、q枚選んで価値がrになるときの最小幅
    for p in range(N):
        for q in range(1,K+1):
            next[q][0] = 0
            for r in range(total+1):

                    
                if B[p] > r:
                    next[q][r] = now[q][r]
                else:
                    next[q][r] = min(now[q][r], now[q-1][r - B[p]] + A[p])
            #dp[0] = copy.deepcopy(dp[1])
        now = next
        next = [[INF for _ in range(total+1)] for _ in range(K+1)]
        next[0][0] = 0
        

    ans = 0
    for i in range(total+1):
        if now[K][i] <= W:
            ans = max(ans,i)
    print(ans)
    #print(dp[0])
    #print(dp[0][0][0])


if __name__ == "__main__":
    main()

In [ ]:
"""
第１２回日本情報オリンピック 予選（オンライン） D

i日目について服jを着た場合に得られる最高得点をdpで探す。
前日の服に依存してi日目の得点が変わる（前日分しか参照しない）ので、昨日と今日の服と得点のの記録だけあればいい
前日にxを来た時の最高得点を全服について持っておいて、当日の更新時に使う形
"""

def main():
    D,N = map(int, input().split())

    T = [int(input()) for _ in range(D)]

    A = [0] * N
    B = [0] * N
    C = [0] * N

    for i in range(N):
        A[i], B[i], C[i] = map(int, input().split())


    # i日目にjを来た時の合計の派手さの絶対値の合計
    #dp = [[-1 for _ in range(N)] for _ in range(D+1)]
    yesterday = [-1] * N
    today = [-1] * N


    # 一日目は何を着ても派手さポイントが得られないので、着れるかどうかだけ判定
    for i in range(N):
        if A[i] <= T[0] <= B[i]:
            yesterday[i] = 0

    # 日数分。一日目は上で決めた
    for i in range(1,D):
        # 服の数だけ
        for j in range(N):
            # i日目に服jが着れる場合
            if A[j] <= T[i] <= B[j]:
                # 前日の服との差分が得点なので確かめる
                for k in range(N):
                    # 前日について、A[k] <= T[i-1] <= B[k]　であれば。
                    # また、i=0の一日目はこのifで全部飛ばされる
                    if yesterday[k] != -1:
                        # k以外の服を着ていた時の方が差が大きいならそのまま、
                        today[j] = max(today[j], yesterday[k] + abs(C[j] - C[k]))
        yesterday = today
        today = [-1] * N
                
    print(max(yesterday))

if __name__ == "__main__":
    main()

In [ ]:
"""
AOJ Longest Common Subsequence

典型的な最長共通部分列問題。
よくあるdp[len(S+1)][len(T+1)]を用意してやると、
pythonだと間に合わなかった（20*1000*1000でO(10^7)のため）
下記解答のように、削れる処理をなるべく削ると通った
制限を含めて結構ためになる問題だと思う
"""



def main():
    N = int(input())
    

    for _ in range(N):
        S = input()
        T = input()

        #dp = [[0 for _ in range(len(T)+1)] for _ in range(len(S)+1)]
        d0 = [0] * (len(T)+1)
        d1 = [0] * (len(T)+1)

        for i,s in enumerate(S):
            d0 = d1[:] # 参照先をかえてコピーのはず
            for j,t in enumerate(T):
                if s == t:
                    d1[j+1] = d0[j] + 1
                elif d1[j+1] < d1[j]:
                    # d1[j+1] = max(d1[j], d0[j+1])の一つ目の場合。
                    # 二つ目の場合はd1を外側のループの度に初期化したりしてないのでもともとその値（d0[j+1]）が入っているから何もしなくていい
                    d1[j+1] = d1[j] 

        print(d1[-1])


if __name__ == "__main__":
    main()

In [ ]:
"""
AtCoder Beginner Contest 113 D

左からj本目の縦棒、上からi進んだところ（以下(i,j)）にいく方法は、
・(i,j-1)から横棒を経由せずにおりてくる
・(i+1,j-1)から横棒を左に移動するのに使って降りてくる
・(i-1, j-1)から横棒を右に移動するのに使って降りてくる

これらを横棒の全パターンについて実施する
また、（i,j）に移動する方法で計算していくのは面倒なので、（i,j）からどこに行けるかでdpするとわかりやすい
実のところ、碁盤の十字の線でスタートから目的点に移動するDPと本質は同じっぽいが、
そこに気付ければめっちゃ難しいという感じでもないか
"""


H,W,K = map(int, input().split())

if W == 1:
    print(1)
    exit()

#dp[i][j]：左からj番目の縦棒で上からiの地点を訪れる場合の数
dp = [[0 for _ in range(W)] for _ in range(H+1)]
#最初は一番左上
dp[0][0] = 1

MOD = 10**9+7
for i in range(H):
    # 横棒のパターンを全探索。横棒は二つの縦棒の間にできるので、W-1箇所入れられるのでそれぞれについてあるORない
    # 1ならそこに横棒がある
    for yoko in range(2**(W-1)):
        if "11" in bin(yoko):
            continue
        
        yoko = bin(yoko)[2:].zfill(W-1)

        for j in range(W):
            #dp[i][j]からどこに行くか考える
            #print(j, len(yoko))
            # 左に行く横棒がある場合
            if j != 0 and  yoko[j-1] == "1":
                dp[i+1][j-1] += dp[i][j]
                dp[i+1][j-1] = dp[i+1][j-1] % MOD

            # 右に行く横棒がある場合
            
            elif j != W-1 and yoko[j] == "1":
                dp[i+1][j+1] += dp[i][j]
                dp[i+1][j+1] = dp[i+1][j+1] % MOD

            # そのまま降りる（横棒がない）場合
            else:
                dp[i+1][j] += dp[i][j]
                dp[i+1][j] = dp[i+1][j] % MOD


print(dp[-1][K-1])

In [ ]:
"""
AOJ DPL_1_C Knapsack Problem

同じ品をいくつでも入れていいので、dpの表の左上（(i-1, j-XXX)）からではなく、左（(i, j-XXX)）から更新するとよい
"""


N,W = map(int, input().split())

vw = []

for i in range(N):
    w,v = map(int, input().split())
    vw.append((w,v))


dp = [[0 for _ in range(W+1)] for _ in range(N+1)]
#i番目までで重さｊを作るときの最大価値
for i in range(N):
    for j in range(W+1):

        if vw[i][1] > j:
            dp[i+1][j] = dp[i][j]
        else:
            dp[i+1][j] = max(dp[i][j], dp[i+1][j - vw[i][1]] + vw[i][0])


print(dp[-1][-1])

## bitDP

dp[i][j]みたいな普通のDPは、だいたい差分が固定で決まっていて（i,jの）、1ステップというか、徐々に状態を変遷させればよかった。

bitDPの場合、状態を集合で捉えて、例えば状態Aと状態Bの論理和の状態に遷移するときの最小コストが〜みたいになるのがうまく理解できていなかった。

ステップ的な変化ではなく、ある状態（状態はbitの集合で表す）から何らか行動を経ることで、別の状態に遷移する。
そこに普通のDPのような連続性というか、初めからゴールまでの一直線な感じがないので、典型的なよくあるDP的思考では捉えにくかった

In [ ]:
"""
ABC 41 - D 徒競走

https://atcoder.jp/contests/abc041/tasks/abc041_d
"""


N,M = map(int, input().split())
xy = [[] for _ in range(N)]

for _ in range(M):
    x,y = map(int, input().split())
    xy[x-1].append(y-1)


# 要素数は起こりうる状態の数で、その状態に至る方法は何通りあるか、を表す
dp = [0] * (1 << N)
# 誰もゴールしていない状態は１通り
dp[0] = 1

# 1が立っている人はゴールしている
for bit in range(1 << N):
    # ゴールしてる人の中でiが最後にゴールしたとする
    for i in range(N):

        # bitの状態でiがゴールしてないなら飛ばす
        if not (bit & (1 << i)):
            continue
        
        
        # iがゴールするまでに、それ以外のゴールしている人たちが、どういう着順であるか、がほしい
        # iが、ゴールしてる人たちより先にゴールしてる、という証言があるとおかしいので飛ばす
        flag = True
        for j in xy[i]:
            if i == j:
                continue
            if bit & (1 << j):
                flag = False

        if flag:
            # i以外の人の着順の場合の数dp[bitのi以外]を足す
            tmp = bit ^ (1 << i)
            dp[bit] += dp[tmp]

print(dp[-1])


## 桁DP

何桁目まで見てるか、そこまでがNのi桁目までより小さいか、後なんか条件　の３要素ぐらいでDPやるとうまくいく


また、現在の状態から、次の状態を作って、それをdp配列の引数に渡すとやりやすい。

逆に、現在の状態は過去のこの状態から～みたいに考えると、沼にはまりやすい

In [ ]:
"""
AtCoder Beginner Contest 007 D - 禁止された数字
https://atcoder.jp/contests/abc007/tasks/abc007_4
"""


A,B = map(str, input().split())

def solve(S):
    N = len(S)
    # i,j,k：i桁目まで見たときに、そこまでがSより厳密に小さい(j=1)、そこまでで4か9を含む(k=1)
    dp = [[[0 for _ in range(2)] for _ in range(2)] for _ in range(N+1)]
    dp[0][0][0] = 1


    for i in range(N):
        digit = int(S[i])
        # 厳密にSのi-1桁目未満であれば1
        for f_less in range(2):
            for f_49 in range(2):
                # 未満ならどの数字もえらべる
                if f_less:
                    d = 9
                else:
                    # Sのi-1桁目まで一致してるなら、次はi桁目未満から選ぶ
                    d = digit
                # nはi桁目に入れる値
                for n in range(d+1):
                    # 次が厳密に小さいになる場合
                    # もとから厳密に小さいか、d未満を選べばSのi桁目までより小さくなる
                    if f_less == 1 or n < d: 
                        f_less_n = 1
                    else:
                        f_less_n = 0

                    # 既に4や9が入っている場合か、あらたに4,9を選ぶ場合
                    if f_49 == 1 or n == 4 or n == 9:
                        f_49_n = 1
                    else:
                        f_49_n = 0

                    """
                    今の状態i,f_less,f_49から、次の状態i+1, f_less_n, f_49_nを作って、今の状態の場合の数を足す感じ
                    """

                    dp[i+1][f_less_n][f_49_n] += dp[i][f_less][f_49]

    return dp[-1][1][1] + dp[-1][0][1]

print(solve(B) - solve(str(int(A) - 1)))


In [ ]:
"""
Typical DP Contest E - 数

https://atcoder.jp/contests/tdpc/tasks/tdpc_number
"""





D = int(input())
N = input()

MOD = 10**9 + 7
l = len(N)
"""
長さ分、Nのi桁目未満か、各桁の数の和をDで割った時の余り（0であれば条件を満たす）
 

"""
dp = [[[0 for _ in range(D)] for _ in range(2)] for _ in range(l+1)]
dp[0][0][0] = 1

for i in range(l):
    digit = int(N[i])
    for f_less in range(2):
        for num_mod in range(D):
            
            # 既にNのi-1桁目までより小さいのでi桁目はどの数字でもOK
            if f_less:
                d = 9
            else:
                # Nのi-1桁目まで一致するのでi桁目はN[i]未満の数字を選ぶ
                d = digit

            for n in range(d+1):
                # i-1桁目が既にNの該当部分までより小さいか、d未満の値であれば、次作る数字もi桁目までより小さい
                if f_less or n < d:
                    f_less_nxt = 1
                else:
                    f_less_nxt = 0

                # numをi桁目に選んだ時に余りがいくらになるか。
                num_mod_nxt = ((num_mod) + n) % D

                dp[i+1][f_less_nxt][num_mod_nxt] += dp[i][f_less][num_mod]
                dp[i+1][f_less_nxt][num_mod_nxt] %= MOD

# 0が条件を満たすので1引く
print(dp[-1][1][0] + dp[-1][0][0] - 1)



In [ ]:
"""
AtCoder Beginner Contest 154 E - Almost Everywhere Zero
https://atcoder.jp/contests/abc154/tasks/abc154_e
"""
N = int(input())
K = int(input())



dp = [[[0 for _ in range(2)] for _ in range(K+1)] for _ in range(len(str(N))+1)]
# 頭から0桁目まで決めて、0以外の数が0個既に使われている時、i桁目までの部分がNより小さいのは0の一通り
dp[0][0][0] = 1


"""
N = 987654として、

（１）3桁目までみたときに、987より小さくなるdp[3][j][1]への派生は
・87から870を作るような、dp[2][j][1]から
・98から980みたいなのがあるから、dp[2][j][0]から。ただし、Nの３桁目が0だと、今作ってる３桁目までの数字98＊が980未満になれないので、その場合はスキップ
↑↑↑ここまではi桁目に0を使うパターン↑↑↑
↓↓↓i桁目に0以外を使うパターン↓↓↓
まだ使える0以外の個数がのこっているとき、
・87から871~879の９パターンに派生する形なので、dp[i][j-1][1] から派生するのが9通りなので * 9 して足す（0以外の値を一つ消化する）
・98から981~986への派生なので、dp[i][j-1][0] * (digit[i] - 1) (例でいうところの７未満の値を使える)　（0以外の値を一つ消化する）

（２）3桁目までみたときに、987と一致するようなdp[3][j][0]への派生は
・もし３桁目が０なら、98から980への派生があるので、dp[i][j][0]から派生できる
・もし３桁目が０ではないのなら、98から987へ派生し、一つ0以外の数字を消化するので、dp[i][j-1][0]から派生


みたいに考えていく。
"""

for i in range(len(str(N))):
    digit = int(str(N)[i])
    for j in range(K+1):
        # i桁目までがNのそこまでより小さい場合、i-1桁目までもNのi-1桁目までより小さいことが確定するから、
        # i-1 -> iへの派生を入れる（from 厳密に小さい to 厳密に小さい）
        dp[i+1][j][1] += dp[i][j][1]

        # i-1桁目までがNのi-1桁目までより小さくなく（一致し）、Nのi桁目が０であれば、作っている数字のi桁目が０になる
        if digit == 0:
            # 厳密に小さくなれないので（from 厳密に小さくない to 厳密に小さくない）
            dp[i+1][j][0] += dp[i][j][0]
        else:
            """
            314159で、31OXXXかつNの3桁目までより小さいパターンは、31XXXXから派生
            """
            # （from 厳密に小さくない（一致する） to 厳密に小さい）
            dp[i+1][j][1] += dp[i][j][0]
        
        # 先頭になんか0以外の数字を入れて、残りがまだ0以外の数字を入れる必要があるとき
        if j-1 >= 0:
            # 0以外のなんかの値を使ったので j-1 -> j かつ  * 9（０以外をいれる）
            dp[i+1][j][1] += dp[i][j-1][1] * 9
            if digit != 0:
                # i桁目が一致する（Nのiまでより小さくならない）パターンについて、直前の場合の数を足す
                dp[i+1][j][0] += dp[i][j-1][0]
                # i桁目までが、Nのそれまでより小さくなるパターンについて、i桁目ではdより小さい値をとれる
                dp[i+1][j][1] += dp[i][j-1][0] * (digit-1)

print(sum(dp[-1][-1]))



## 区間DP


配列の隣り合うN個についてXXXな処理を繰り返したときの、最大のOOOを求めよ、みたいな問題に使う。  

区間[i,j)に対する最大の回数をdp(i,j)とする。LとRを選ぶというより、  
 - 区間の幅
 - 区間の始まりのインデックス
 
の二つでループを回して、最終的に全体の答えを導く方が個人的にはやりやすい。  



In [ ]:
#　典型的なパターン

# 区間幅
for length in range(N):
    # 区間の始まりのインデックス
    for start in range(N-length):
        last = start + length
        """
        この区間について、dp[start][last]が更新されうるパターンは、
        ・dp[start]][last] = dp[start][k] + dp[k+1][last]として更新できるiとjにあるkが存在する場合
        ・[start+1,last-1)が条件を満たして消え、startとlastについても条件を満たす場合
        の二通り。例えばiwiの問題であれば、[start+1,last-1)の間の文字がすべて消えるので、その両端でさらにiwiを消せる、みたいな
        """

        for k in range(start + 1, last - 1):
            dp[start][last] = max(dp[start][last], dp[start][k] + dp[k+1][last])


        if dp[start][last] == OOO:
            # その両端の幅の分だけ何らかの比較
            for i in range(X):









In [ ]:
"""
AOJ 1611 Daruma Otoshi

TLEになるけどロジックはあっているはず。


"""





while True:
    N = int(input())
    if N == 0:
        exit()
    A = list(map(int,input().split()))

    dp = [[0 for _ in range(N+1)] for _ in range(N+1)]

    # 区間の幅（みるダルマの数）
    for length in range(2,N+1):
        for start in range(N - length + 1):
            last = start + length

            # start ~ lastが、start~k-1, k~lastに分解してみると更新できる場合があるか確認
            for k in range(start+1, last-1):
                dp[start][last] = max(dp[start][last], dp[start][k] + dp[k][last])

            
            # start+1 ~ last-1が全部落とし切れて、そのあとできる隣接するstartとlastのダルマの差が1以下の場合、これらも落とせる
            if dp[start+1][last-1] == length-2:
                if abs(A[start] - A[last - 1]) <= 1:
                    dp[start][last] = max(dp[start][last], length)
                else:
                    dp[start][last] = max(dp[start][last], length - 2)

    print(dp[0][-1])


In [ ]:
"""
AOJ 1611 Daruma Otoshi

こっちは間に合う


"""
def main():
    while True:
        N = int(input())
        if N == 0:
            exit()
        A = list(map(int,input().split()))
        can_drop = [[False] * N for _ in range(N)]
        # d[i]: i個のブロック列で最大何個落とせるか
        d = [0] * (N+1)

        # 隣接するのが落とせるかどうか
        for i in range(N-1):
            if abs(A[i] - A[i+1]) <= 1:
                can_drop[i][i+1] = True

        # 何個分をみるか。一個増えても落とせるやつ個数が増えたりしないので２刻み
        for i in range(3, N, 2):
            # どのブロックはじめで見るか。
            for j in range(N-i):
                # 間
                last = j + i
                for k in range(j+1, j+i):
                    # j~lastまでで落とせないが、j~k, k+1~lastに分けてみると、両方の部分で結果的にj~lastで落とせる場合
                    if not can_drop[j][last] and can_drop[j][k] and can_drop[k+1][last]:
                        can_drop[j][last] = True
                        break
                # j ~ lastで落ちないが、j+1~last-1で落とせて、jとlastの差が１以下の場合は、j~lastで落とせると言える
                if not can_drop[j][last] and can_drop[j+1][last-1] and abs(A[j] - A[last]) <= 1:
                    can_drop[j][last] = True

        # can_dropのTrueが続いている最長部分を探す
        for i in range(N):
            for j in range(N):
                if can_drop[j][i]:
                    d[i] = max(d[i], d[j-1] + i - j + 1)
        
            d[i] = max(d[i], d[i - 1])
        print(d[N-1])        

if __name__ == "__main__":
    main()
